# Timestream Processing and Mapmaking with SPT Data

In this notebook, we'll be looking at some data from the South Pole Telescope using the [spt3g_software](https://github.com/CMB-S4/spt3g_software) code.  To get started, let's first download and install all of the necessary components.  We'll be using pre-build wheels of the software compiled by the Simons Observatory collaboration with their [`so3g` package](https://github.com/simonsobs/so3g), but using only the `spt3g.core` library that it includes.

In [ ]:
# install required packages
!python -c "import spt3g" || pip install so3g

# download dataset
!ls spt3g_tutorial_data || ( \
    wget https://sptlocal.grid.uchicago.edu/~arahlin/cmb_school_2024/spt3g_tutorial_data.tar.gz && \
    tar xzvf spt3g_tutorial_data.tar.gz \
)

While the above cell is executing (the download may take some time), spend a few minutes familiarizing yourself with the [documentation](https://cmb-s4.github.io/spt3g_software/).  Today, we'll be covering the following topics:

- how to work with G3Frames and G3FrameObjects, and combining them into pipelines
- how to use G3Units
- how to work with detector calibration data
- the basics of timestream processing and working with G3TimestreamMap objects
- simple (filter-and-bin) mapmaking, which combines detector timestreams into a map on the sky

Once the installation is complete, let's make sure the package works, and import all the necessary libraries into the python environment:

In [ ]:
try:
    import spt3g
except ImportError:
    import so3g
from spt3g import core
from spt3g.core import G3Units as U
import numpy as np
import matplotlib.pyplot as plt

## G3 Data Format

The G3 file format stores data into "frames", which behave much like python `dict`s, with a few additional features.  Let's start by reading in a set of static calibration data for a subset of SPT detectors.  One way to load a file from disk is to use the `G3File` class.  When initialized with a filename, this object behaves like a generator, which returns a frame when looped over.  We happen to know that this particular file contains only a single frame, so we can convert the generator into a list and return the first frame:

In [ ]:
fr = list(core.G3File("spt3g_tutorial_data/calibration.g3"))[0]
print(fr)

This frame contains several keys.  Let's take a look at the `DetectorCalibration` object first.  This is a mapping (i.e. a `dict`), keyed by detector name, of calibration constants for each detector in the dataset.

In [ ]:
cal = fr["DetectorCalibration"]
print(cal)
# print the first 10 keys
print(cal.keys()[:10])
# print the calibration for one detector
print(cal["2019.00e"])

## Units

All unitful data are stored ["in G3Units"](https://cmb-s4.github.io/spt3g_software/units.html), meaning that they have been multiplied by some constant, whose value you do not need to know.  To convert a value in G3Units to one with physical meaning, simply divide by the units you want.  For example, let's plot the detector offsets for all of the channels in this dataset.  The detectors in the array lie on a so-called focal plane inside the telescope, so neighboring detectors do not necessarily see light rays from the same point on the sky as the telescope moves -- their pointing is shifted relative to that of the effective center, or "boresight" of the telescope.

In [ ]:
xoff = fr["DetectorXOffset"]
yoff = fr["DetectorYOffset"]

plt.plot(np.asarray(xoff.values()) / U.arcmin, np.asarray(yoff.values()) / U.arcmin, ".", ms=1)
plt.gca().set_aspect("equal")
plt.xlabel("X Offset [arcmin]")
plt.ylabel("Y Offset [arcmin]");

Another entry in our calibration frame is called `DetectorCalibrationUnits`, which defines the *family* of units in which detector timestreams end up when the appropriate calibration constants are applied.  The `core.G3TimestreamUnits` enum object contains the various families of units that are available.  What units family are the calibration constants stored in?

In [ ]:
# print the mapping from an integer value to a units family name
print(core.G3TimestreamUnits.values)
cal_units = core.G3TimestreamUnits.values[fr["DetectorCalibrationUnits"]]
print(cal_units)

## Modules and Pipelines

A common way of manipulating SPT data is to write `modules`, which are functions or classes that take a `G3Frame` as an argument and manipulate it.  We can combine multiple modules together into a `pipeline` to analyze a stream of frames in sequence.  For more details on how this works, see the [documentation](https://cmb-s4.github.io/spt3g_software/modules.html).

For example, here is a simple pipeline module that reads our timestream data and prints it out:

In [ ]:
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename="spt3g_tutorial_data/timestreams.g3")
pipe.Add(core.Dump)
pipe.Run()

You'll notice that the input data file contains many `Scan` frames, which themselves contain telescope boresight pointing timestreams, as well as the raw detector timestreams, all sampled at the same data rate.  The SPT scans the sky in azimuth, stepping in elevation between back-and-forth sweeps.  Let's see if we can plot up the scan pattern.  We can define a function that only works on scan frames, and insert it into our pipeline:

In [ ]:
def PlotRaDec(frame):
    # skip any frame that doesn't contain the right key
    if "BoresightRa" not in frame:
        return

    # plot coordinates in real units
    plt.plot(frame["BoresightRa"] / core.G3Units.deg, frame["BoresightDec"] / core.G3Units.deg)

In [ ]:
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename="spt3g_tutorial_data/timestreams.g3")
pipe.Add(PlotRaDec)
pipe.Run()

plt.xlabel("Right Ascension [deg]")
plt.ylabel("Declination [deg]");

You'll notice that the timestreams are split into sections of constant declination, and sections where the telescope is turning around and/or stepping in declination.  We can add a module to exclude these so-called "turnaround" frames, so that we are only keeping the data where the telescope is moving smoothly in one direction:

In [ ]:
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename="spt3g_tutorial_data/timestreams.g3")
pipe.Add(lambda fr: fr.get("Turnaround", False) == False)
pipe.Add(PlotRaDec)
pipe.Run()

We can also manipulate the data in the frames.  Once the frames are updated, the next module in the pipeline will receive the frame with the updated data, since this manipulation is effectively happening "in-place".  Let's define a function to apply the calibration constants we loaded above to the input `RawTimestreams`.  This will convert the raw detector timestreams from ADC units to units of CMB temperature.  Note that the modified timestreams are stored to a new key in the frame, because the existing key cannot be overwritten.

In [ ]:
def Calibrate(frame):
    # skip frames that don't contain the input key
    if "RawTimestreams" not in frame:
        return

    # get the input timestream data
    ts_in = frame["RawTimestreams"]
    
    # create a new output timestream object
    ts_out = core.G3TimestreamMap()

    # loop over timestreams and multiply by the calibration constant for each detector
    for det, ts in ts_in.items():
        ts2 = ts.astype(float) * cal[det]
        ts2.units = cal_units  # update units
        ts_out[det] = ts2

    # store the calibrated timestreams to the output key in the frame
    frame["CalTimestreams"] = ts_out

## Making a sky map

Let's take the data for a single detector and create a sky map from it.  First, we need to define the pixel grid onto which the timestream data will be binned.  To make the math relatively easy, let's stick with Cartesian coordinates for the time being:

In [ ]:
# center of the sky map
ra0 = 8.0 * U.rahour + 59.0 * U.raminute + 4.8 * U.rasecond
dec0 = -(47.0 * U.degrees + 30.0 * U.arcmin + 36.0 * U.arcsec)

# map dimensions
xlen = 0.5 * U.deg
ylen = 0.5 * U.deg

# pixel resolution
res = 0.5 * U.arcmin

# number of bins along each axis
nx = int(xlen / res)
ny = int(ylen / res)

# bin edges
ra_edges = np.linspace(-xlen / 2, xlen / 2, nx + 1) + ra0
dec_edges = np.linspace(-ylen / 2, ylen / 2, ny + 1) + dec0

And now we need to define a module for binning the timestream data into this sky map.  We're going to iteratively update the sky map with each input frame that we read from file, so the binner cannot be a simple function.  We define a class, in which we store all of the input arguments and the output data arrays as attributes of the class.  Its `__call__` method is then used in the pipeline to update the data arrays.

Notice that we are including pointing offsets for each detector.  The pointing offsets are defined relative to the telescope boresight (i.e. the effective center of the focal plane), so must be converted to sky offsets to be applied correctly.

In [ ]:
class SingleMapBinner:
    def __init__(self, det, timestreams="CalTimestreams"):
        self.det = det
        self.timestreams = timestreams

        # array for storing the binned timestream data
        self.data = np.zeros((ny, nx), dtype=float)

        # array for storing the number of times each pixel is "hit" in the timestreams
        self.hits = np.zeros((ny, nx), dtype=float)
    
    def __call__(self, frame):
        if self.timestreams not in frame:
            return

        if self.det not in frame[self.timestreams]:
            return

        ts = frame[self.timestreams][self.det]

        # calculate offset pointing
        dx = xoff[self.det] / np.cos(dec0 / U.rad)
        dy = yoff[self.det]
        x = np.asarray(frame["BoresightRa"]) + dx
        y = np.asarray(frame["BoresightDec"]) - dy

        # update data and hits, in-place
        self.data += np.histogram2d(y, x, bins=[dec_edges, ra_edges], weights=ts)[0]
        self.hits += np.histogram2d(y, x, bins=[dec_edges, ra_edges])[0]

To use this module object, we first instantiate it, then add the instance to the pipeline.  This allows access to the updated `data` and `hits` attributes when the pipeline is finished running.

In [ ]:
# instantiate binner
binner = SingleMapBinner(det="2019.00e")

# run pipeline
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename="spt3g_tutorial_data/timestreams.g3")
pipe.Add(lambda fr: fr.get("Turnaround", False) == False)
pipe.Add(Calibrate)
pipe.Add(binner)
pipe.Run()

m = binner.data / binner.hits / U.mK
plt.imshow(m)
plt.colorbar(label="Temperature [mK]");

Notice that this map is not very well populated.  It seems that our one detector is barely in view of the field we're looking at.

<span style="color:red">EXERCISE:</span> Modify the map binner above to loop over all detectors.  Call it `MapBinner`, and run a new pipeline to plot up the resulting total sky map.

In [ ]:
# your code and plots go here

This map binning scheme is quite simple and relatively inefficient.  The [spt3g.maps](https://cmb-s4.github.io/spt3g_software/moddoc_maps.html) package includes definitions of sky map objects that are able to handle different sky projections and pixelization schemes, as well as tools for binning timestreams to sky maps (and vice versa) in an efficient way.

This tutorial also does not touch on detector polarization or polarized mapmaking; for more details on how this works, see, for example [this pedagogical paper](https://arxiv.org/abs/astro-ph/0606606).

## Timestream Processing

We now have a calibrated map, but it has some strange striping and large scale drifts in it, when we would have expected to see some kind of compact blob in the image.  This is because the detector timestreams drift on long timescales due to various forms of contamination: emission from our own atmosphere, thermal drifts in the internal cryogenic environment of the camera, etc.  To remove these drifts, we need to filter the timestreams in such a way as to remove these long timescale drifts.

<span style="color:red">EXERCISE:</span> Create a new pipeline module called `MeanFilter` that subtracts the mean value of each timestreams for each scan, and stores the resulting timestreams to a new `"FilteredTimestreams"` key in the scan frame.  Write a new pipeline to bin these filtered timestreams into a map.  What does the sky map look like with this coarse filter applied?  Zoom in on the color scale (e.g. by setting the `vmax` argument for `imshow`).  What features do you see in the map?

In [ ]:
# your code and plots go here

This particular image has a large and bright astrophysical source in the middle.  High-resolution telescopes like ACT and SPT are able to resolve such sources all over the sky.  However, notice that simple filtering without accounting for the existence of the source in the data results in pronounced "wings" along the telescope's scan direction that extend far beyond the width of the source itself.  To reduce the presence of these wings, we can mask out the source prior to applying a filter.

<span style="color:red">EXERCISE:</span> Modify your `MeanFilter` module to compute the timestream mean outside of an 8 arcmin radius around the source.  Call this new module `MaskedMeanFilter`.  Don't forget to account for the individual detector offsets in constructing your mask.

In [ ]:
# your code and plots go here

Detector data can be filtered and processed in many more complicated ways.  For example, we could subtract a higher-order polynomial to filter out some smaller scale modes from the data.  We could also use a Fourier filter to do more complex filtering, such as notching out particular lines or limiting the frequency bandwidth of the timestreams.  A complete low-level analysis pipeline incorporates a variety of filters and data cuts in order to reconstruct the underlying sky with high signal-to-noise.